In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join("..", "src")))


In [2]:
import os

In [3]:
%pwd

'e:\\D-drive\\Gen AI\\Projects\\ChickenDisease-Detection-DeepLearning\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\D-drive\\Gen AI\\Projects\\ChickenDisease-Detection-DeepLearning'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    bucket_name: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
%pwd

'e:\\D-drive\\Gen AI\\Projects\\ChickenDisease-Detection-DeepLearning'

In [9]:
!pip install pyyaml



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install joblib

  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install -r requirements.txt

Obtaining file:///E:/D-drive/Gen%20AI/Projects/ChickenDisease-Detection-DeepLearning (from -r requirements.txt (line 14))
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.12.0 (from versions: 2.20.0rc0, 2.20.0)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow==2.12.0


In [11]:
!pip install ensure



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from cnnimgClassifier.constants import *
from cnnimgClassifier.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            bucket_name=config.bucket_name,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [15]:
!pip install boto3 

  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/14.1 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/14.1 MB 10.5 MB/s eta 0:00:02
   ------------- -------------------------- 4.7/14.1 MB 11.3 MB/s eta 0:00:01
   -------------------- ------------------- 7.3/14.1 MB 11.5 MB/s eta 0:00:01
   --------------------------- ------------ 9.7/14.1 MB 11.5 MB/s eta 0:00:01
   --------------------------------- ------ 11.8/14.1 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 14.1/14.1 MB 10.9 MB/s eta 0:00:00
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)

   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [u


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
!pip install dotenv


   ---------------------------------------- 0/2 [python-dotenv]
   ---------------------------------------- 0/2 [python-dotenv]
   ---------------------------------------- 0/2 [python-dotenv]
   ---------------------------------------- 2/2 [dotenv]




[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import boto3
import os
from dotenv import load_dotenv
from botocore.exceptions import ClientError
import zipfile
from cnnimgClassifier import logger
from cnnimgClassifier.utils.common import get_size
import tempfile

load_dotenv()

False

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def _make_s3(self, region_name, aws_access_key_id, aws_secret_access_key):
        return boto3.client(
            "s3",
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )

    def _stream_download(self, s3, bucket_name, object_key, download_path):
        # Ensure target directory exists
        Path(download_path).parent.mkdir(parents=True, exist_ok=True)

        print(f"Downloading {object_key} from bucket {bucket_name} to {download_path}...")
        response = s3.get_object(
            Bucket=bucket_name,
            Key=object_key,
            RequestPayer="requester",
        )
        body = response["Body"]

        # Write to temp file, then atomic rename
        tmp_dir = Path(download_path).parent
        with tempfile.NamedTemporaryFile(dir=tmp_dir, delete=False) as tmp_f:
            tmp_path = tmp_f.name
            for chunk in body.iter_chunks(chunk_size=8 * 1024 * 1024):
                if chunk:
                    tmp_f.write(chunk)

        body.close()
        os.replace(tmp_path, download_path)
        print("Download complete.")

    def download_from_s3(
        self,
        bucket_name,
        object_key,
        download_path,
        aws_access_key_id,
        aws_secret_access_key,
        region_name="us-east-1",
    ):
        s3 = self._make_s3(region_name, aws_access_key_id, aws_secret_access_key)

        try:
            self._stream_download(s3, bucket_name, object_key, download_path)

        except ClientError as e:
            code = e.response["Error"].get("Code", "")
            msg = e.response["Error"].get("Message", str(e))
            print(f"Error ({code}): {msg}")

            if code in ["403", "AccessDenied"]:
                print("→ Check IAM permissions and ensure RequestPayer='requester' is allowed in the bucket policy.")
                raise

            elif code in ["PermanentRedirect", "301", "AuthorizationHeaderMalformed"]:
                try:
                    loc = s3.get_bucket_location(Bucket=bucket_name)["LocationConstraint"]
                    retry_region = loc or "us-east-1"
                    print(f"→ Detected bucket region: {retry_region}. Retrying download...")
                    s3_retry = self._make_s3(retry_region, aws_access_key_id, aws_secret_access_key)
                    self._stream_download(s3_retry, bucket_name, object_key, download_path)
                except ClientError as e2:
                    code2 = e2.response["Error"].get("Code", "")
                    msg2 = e2.response["Error"].get("Message", str(e2))
                    print(f"Retry failed ({code2}): {msg2}")
                    raise
            else:
                raise

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory specified by self.config.unzip_dir.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    bucket_name = "chicken-data-2025"
    object_key = "data.zip"
    download_path = data_ingestion_config.local_data_file  # already set in config

    data_ingestion.download_from_s3(
        bucket_name,
        object_key,
        download_path,
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region_name="us-east-1",
    )

    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-11-03 13:02:41,265: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-03 13:02:41,269: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-03 13:02:41,272: INFO: common: created directory at: artifacts]
[2025-11-03 13:02:41,275: INFO: common: created directory at: artifacts/data_ingestion]


NoCredentialsError: Unable to locate credentials